# Albedo Afforestation

In [ ]:
import pandas as pd
import numpy as np
import glob

In [ ]:
# Data location
project_path = './'
data_path = project_path + '../data/'

data_fn = data_path + 'dataset.csv'
afforestation_fn = data_path + 'afforestation_albedo.csv'

In [ ]:
def load_data(fn, index_col=False, silent=False):
    if (not silent):
        print('-', fn.split('/')[-1])
    temp = pd.read_csv(fn, index_col=index_col)
    temp['DateTime'] = pd.to_datetime(temp['DateTime'], format='%Y-%m-%d %H:%M:%S', utc=True)
    return(temp)

def weighted_avg_and_std(values, weights):
    values = np.ma.masked_invalid(values)
    #masked_arr = np.ma.masked_array(values, np.isnan(values))
    
    # Return the weighted average and standard deviation.
    average = np.ma.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.ma.average((values-average)**2, weights=weights)
    
    maximum = np.max(values)
    minimum = np.min(values)
    return (average, np.sqrt(variance), maximum, minimum)

In [ ]:
print('Loading data...')

df = load_data(data_fn)

desert_df = df.loc[df['Ecosystem'] == 'Afforestation desert background'].copy()

forest_df = df.loc[(df['Ecosystem'] == 'Afforestation') & ((df['DateTime'].dt.year == 2018) | (df['DateTime'].dt.year == 2019))].copy()
forest_long_duration = load_data(afforestation_fn)

print('Done...')

In [ ]:
print('Calculating mean albedo:')

#print(desert_df.columns.values)

print('  - Calculate albedo')
# Calculate albedo
forest_df['albedo'] = forest_df['SWout'] / forest_df['SWin']
desert_df['albedo'] = desert_df['SWout'] / desert_df['SWin']

# Filter out values divided by 0
# Forest
forest_df = forest_df.loc[forest_df['albedo'] > 0].copy()
forest_df = forest_df.loc[(forest_df['SWin'] > 0) & (forest_df['SWout'] > 0)].copy()
forest_df.loc[forest_df['albedo'] > 1, 'albedo'] = np.nan
# Desert
desert_df = desert_df.loc[desert_df['albedo'] > 0].copy()
desert_df = desert_df.loc[(desert_df['SWin'] > 0) & (desert_df['SWin'] > 0)].copy()
desert_df.loc[desert_df['albedo'] > 1, 'albedo'] = np.nan


print('  - Forest albedo (based on 2018-2019)')
mean, std, maximum, minimum = weighted_avg_and_std(forest_df['albedo'], forest_df['SWin'])
print('    - Mean: ', str(np.round(mean, 2)) + '±' + str(np.round(std, 3)))
print('    - Max.: ', str(np.round(maximum, 2)))
print('    - Min.: ', str(np.round(minimum, 2)))
mean_forest = mean
std_forest = std

print('  - Desert albedo')
mean, std, maximum, minimum = weighted_avg_and_std(desert_df['albedo'], desert_df['SWin'])
print('    - Mean: ', str(np.round(mean, 2)) + '±' + str(np.round(std, 3)))
print('    - Max.: ', str(np.round(maximum, 2)))
print('    - Min.: ', str(np.round(minimum, 2)))
mean_desert = mean
std_desert = std

print('  - Change in albedo from planting a forest')
delta_albedo = mean_desert - mean_forest
std_delta_albedo = np.sqrt(std_desert**2 + std_forest**2)
print('    - Mean: ', str(np.round(delta_albedo, 2)) + '±' + str(np.round(std_delta_albedo, 3)))

print('Done...')

### Yatir afforestation 2010-2020

In [ ]:
forest_df = forest_long_duration

print('Calculating annual albedo:')

print('  - Calculate albedo')
# Calculate albedo
forest_df['albedo'] = forest_df['SWout'] / forest_df['SWin']
desert_df['albedo'] = desert_df['SWout'] / desert_df['SWin']

# Filter out values divided by 0
# Forest
forest_df = forest_df.loc[forest_df['albedo'] > 0].copy()
forest_df = forest_df.loc[(forest_df['SWin'] > 0) & (forest_df['SWout'] > 0)].copy()
forest_df.loc[forest_df['albedo'] > 1, 'albedo'] = np.nan
# Desert
desert_df = desert_df.loc[desert_df['albedo'] > 0].copy()
desert_df = desert_df.loc[(desert_df['SWin'] > 0) & (desert_df['SWin'] > 0)].copy()
desert_df.loc[desert_df['albedo'] > 1, 'albedo'] = np.nan

print('  - Forest albedo (based on 2010-2020)')
mean, std, maximum, minimum = weighted_avg_and_std(forest_df['albedo'], forest_df['SWin'])
print('    - Mean: ', str(np.round(mean, 2)) + '±' + str(np.round(std, 3)))
print('    - Max.: ', str(np.round(maximum, 2)))
print('    - Min.: ', str(np.round(minimum, 2)))
mean_forest = mean
std_forest = std

print('  - Desert albedo')
mean, std, maximum, minimum = weighted_avg_and_std(desert_df['albedo'], desert_df['SWin'])
print('    - Mean: ', str(np.round(mean, 2)) + '±' + str(np.round(std, 3)))
print('    - Max.: ', str(np.round(maximum, 2)))
print('    - Min.: ', str(np.round(minimum, 2)))
mean_desert = mean
std_desert = std

print('  - Change in albedo from planting a forest')
delta_albedo = mean_desert - mean_forest
std_delta_albedo = np.sqrt(std_desert**2 + std_forest**2)
print('    - Mean: ', str(np.round(delta_albedo, 2)) + '±' + str(np.round(std_delta_albedo, 3)))

print('Done...')